In [ ]:
#include <string>
#include <set>
#include <map>
#include <iostream>
#include "infoapi.hpp"

using namespace std;

class TicketsObject: public DataObject{
public:
    TicketsObject(unsigned pid, EventLog&log):DataObject(pid,log),
        get_ticket_transaction(*this){
            for(int n=1; n<=5; n++) vacant_tickets.insert(n);
        }
public:
    bool get_ticket(const string&name,int&ticket_number){
        
        get_ticket_transaction.name = name;
        get_ticket_transaction.rand_num = rand();
        
        get_ticket_transaction.run();

        if(selected_tickets.find(name)==selected_tickets.end()) return false;
        
        ticket_number=selected_tickets[name];
        return true;
    }
    void print_tickets(){
        update();
        for(auto& t: selected_tickets){
            cout << "         Name:" << t.first  << endl
                 << "Ticket number:" << t.second << endl << endl;
        }
    }
private:
    class GetTicket: public Transaction{
    public:
        GetTicket(TicketsObject&td):Transaction(td),data(td){}
    public:
        string name;
        int    rand_num;
    private:
        void on_run()override{
            if(data.vacant_tickets.size()==0 ||
                data.selected_tickets.find(name)!=data.selected_tickets.end()) return;
                
            int selected = rand_num % data.vacant_tickets.size();
            set<int>::iterator it = data.vacant_tickets.begin();
            for(int i=0; i!=selected; i++,it++);
            int ticket_number = *it;
          
            data.vacant_tickets.erase(ticket_number);    
            data.selected_tickets[name] = ticket_number;
        }
    private:
        TicketsObject& data;
        void on_save(ostream&out)override{ out << rand_num << endl << name;  }
        void on_load(istream&in)override{  in >> rand_num >> name; }

    } get_ticket_transaction;

public:
    set<int>        vacant_tickets;
    map<string,int> selected_tickets;
};

In [ ]:
EventLog event_log;

In [ ]:
{// Getting a ticket.
    TicketsObject tickets_object(0,event_log);
    
    string name;
    int ticket_number;
    
    cout << "Please, print your name:" << endl; 
    cin >> name;
    
    if(tickets_object.get_ticket(name,ticket_number))
        cout << "Your ticket number is:" << ticket_number << endl;
    else
        cout << "You're late. There are no tickets left." << endl;     
}

In [ ]:
{//Printing all tickets.
    TicketsObject tickets_object(1,event_log);
    tickets_object.print_tickets();
}